In [1]:
import numpy as np
import scipy as sp
%load_ext autoreload
%autoreload 2

# simulate data y~n(mu,sigma), mu~n(mu0,sig0)

In [31]:
p = 2
rho = 0.5
mu0 = np.ones([p,1])
sigma0 = np.identity(p)
sigma = np.array([[1,rho],[rho,1]])
n = 100
y = np.zeros([p,n])
for i in range(n):
    mn = np.random.multivariate_normal(mu0.ravel(),sigma0)
    obs = np.random.multivariate_normal(mn.ravel(),sigma)
    y[:,i] = obs

# get posterio distribution, analytically

In [32]:
ybar = np.reshape(np.mean(y,axis=-1),(2,1))
post_sig = np.linalg.inv((n*np.linalg.inv(sigma))+np.linalg.inv(sigma0))

def get_post_mean(mu0,sigma0,sigma,ybar,n):
    tmp1 = np.matmul(np.linalg.inv(sigma0),mu0)
    tmp2 = n*np.matmul(np.linalg.inv(sigma),ybar)
    tmp3 = np.matmul(post_sig,tmp1+tmp2)
    return tmp3
    
def get_rho(sigma):
    return sigma[1,0]/(np.sqrt(sigma[0,0])*np.sqrt(sigma[1,1]))


#post_sig = np.linalg.inv((n*np.matmul(np.linalg.inv(sigma),ybar))+np.linalg.inv(sigma0))
post_mean = get_post_mean(mu0,sigma0,sigma,ybar,n)
post_rho = get_rho(post_sig)
prior_rho = get_rho(sigma0)

print(post_mean)
print(get_rho(post_sig))
print(get_rho(sigma))

[[1.03735823]
 [1.17609898]]
0.49627791563275436
0.5


# estimate via Deep QR

# estimate posterior over MCMC using PyStan

In [33]:
import pystan
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set()  # Nice plot aesthetic
np.random.seed(101)

model = """
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] y;
}
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}
model {
    y ~ normal(alpha + beta * x, sigma);
}
"""
# Parameters to be inferred
alpha = 4.0
beta = 0.5
sigma = 1.0

# Generate and plot data
x = 10 * np.random.rand(100)
y = alpha + beta * x
y = np.random.normal(y, scale=sigma)
# Put our data in a dictionary
data = {'N': len(x), 'x': x, 'y': y}

# Compile the model
sm = pystan.StanModel(model_code=model)

# Train the model and generate samples
fit = sm.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=101)
summary_dict = fit.summary()
print(summary_dict)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cb4cc9c2a04d0e34d711077557307fb7 NOW.


OrderedDict([('summary', array([[ 3.84723296e+00,  7.79091576e-03,  2.04476226e-01,
         3.43219865e+00,  3.71204470e+00,  3.84394529e+00,
         3.98862143e+00,  4.24730697e+00,  6.88824561e+02,
         1.00380551e+00],
       [ 5.18992255e-01,  1.35712972e-03,  3.57738839e-02,
         4.51553674e-01,  4.94031077e-01,  5.19022037e-01,
         5.42252166e-01,  5.92751491e-01,  6.94847442e+02,
         1.00513724e+00],
       [ 1.01833638e+00,  2.22970579e-03,  7.24110929e-02,
         8.89964491e-01,  9.66515269e-01,  1.01535241e+00,
         1.06287782e+00,  1.17144063e+00,  1.05466633e+03,
         1.00353560e+00],
       [-5.06707612e+01,  4.66894179e-02,  1.19914649e+00,
        -5.37411249e+01, -5.11806548e+01, -5.03810091e+01,
        -4.98233326e+01, -4.92740575e+01,  6.59640882e+02,
         1.00925428e+00]])), ('c_summary', array([[[ 3.82870373e+00,  2.07959206e-01,  3.85228783e+00,
          2.07291473e-01],
        [ 3.85552278e+00,  2.09997480e-01,  3.85241749e+00,